Single Step environments are the most widespread type of environment when doing RL with an LLM as policy.

It's called *single step* environment, because the agent only does one step. It gets a question sampled from the dataset (the initial state / observation) and then answers. The answer is then scored according to the reward function. Recently, rules-based reward functions, i.e. functions without any learnable parameters, have been successfully used to do RL with LLMs as as policy.

Since many RL algorithms (such as GRPO) need multiple rollouts at each step, batching is important to guarantee concurrency / parallelism. This notebook will show how to use batched environments.

First, we have to load a dataset from which we will sample questions. The dataset can be either a `StaticDataset`, which is finite and the length is known at runtime, or it can be a `BaseGenerator`, which is an infinite supply of question - answer pairs, synthetically generated in some way (depending on the implementation).

We already have a simple script where we use a `StaticDataset`. In here, we will use the `FewShotGenerator` which will generate us new DataPoints on the fly.

First, we need a Seed Dataset for the `FewShotGenerator` to sample from. For this, let's create a `StaticDataset`:

In [ ]:
from camel.datasets import StaticDataset
from camel.logger import get_logger

logger = get_logger(__name__)

raw_data = [
    {
        "question": "Evaluate the limit as x approaches 0 of (sin(3*x) - 3*x) / x**3.",  # noqa: E501
        "final_answer": "-9/2",
        "rationale": '''from sympy import symbols, limit, sin
x = symbols('x')
expr = (sin(3*x) - 3*x) / x**3
result = limit(expr, x, 0)
print(result)''',
    },
    {
        "question": "Evaluate the definite integral of (1 - x**2)**3 from x = 0 to x = 1.",  # noqa: E501
        "final_answer": "16/35",
        "rationale": '''from sympy import symbols, integrate
x = symbols('x')
expr = (1 - x**2)**3
result = integrate(expr, (x, 0, 1))
print(result)''',
    },
    {
        "question": "Evaluate the limit as n approaches infinity of n*(sqrt(n**2 + 1) - n).",  # noqa: E501
        "final_answer": "1/2",
        "rationale": '''from sympy import symbols, limit, sqrt
n = symbols('n', positive=True)
expr = n*(sqrt(n**2 + 1) - n)
result = limit(expr, n, float("inf"))
print(result)''',
    },
    {
        "question": "Compute the sum of the series sum from n = 1 to 50 of 1/(n*(n+1)).",  # noqa: E501
        "final_answer": "50/51",
        "rationale": '''from sympy import symbols, summation
n = symbols('n', positive=True, integer=True)
expr = 1/(n*(n+1))
result = summation(expr, (n, 1, 50))
print(result)''',
    },
]

seed_dataset = StaticDataset(raw_data)

The `FewShotGenerator` needs an python interpreter to compute a pseudo ground truth from the code it generated. For this, let's define a `PythonVerifier`.

In [ ]:
from camel.verifiers import PythonVerifier

verifier = PythonVerifier(required_packages=["sympy"])
await verifier.setup(uv=True)

Lastly, we need a Model to generate the synthetic datapoints. Let's use the `ModelFactory`.

Note: We use GPT-4o Mini as a default here, feel free to use other models!

In [ ]:
from getpass import getpass
import os

openai_api_key = getpass('Enter your OpenAI API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig
from camel.datasets import FewShotGenerator

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(),
)

# Note: When the generator needs to create new datapoints, it will on default create 20 new datapoints
# Since we are paying for the API, let's set this number to 2 instead
generator = FewShotGenerator(
    puffer=2, seed_dataset=seed_dataset, verifier=verifier, model=model
)

Now that our generator is all set up, let's create a `SingleStepEnv`.

We can then call `env.reset()` to draw from the initial state distribution and return an observation, which can then be fed into the agent.

In [ ]:
from camel.environments import Action, SingleStepEnv

env = SingleStepEnv(generator, verifier)

obs = await env.reset(seed=42)

print(obs)

The agent would then process this observation and select an action, which it would feed into the `step` function. For this, we will make use of *extractor*. An extractor takes the LLM response and extracts the verifiable part out of it. Extractors can be initialized with different strategies which modifies the extraction behavior. We then compare it to our computed pseudo ground truth which we got internally by running the code in the rationale with the `PythonVerifier`

In [ ]:
from camel.agents import ChatAgent
from camel.extractors import BaseExtractor, BoxedStrategy

# Initialize extractor
extractor = BaseExtractor([[BoxedStrategy()]])
await extractor.setup()
agent = ChatAgent(model=model)

SYSTEM_PROMPT = r"""System Prompt:
You are a helpful AI designed to answer mathematical questions with clarity and precision. Your task is to provide a step-by-step explanation for 
any mathematical problem posed by the user, ensuring the response is easy to follow. Adhere to these guidelines:
Analyze the mathematical question carefully and break down the solution process into clear, logical steps.
Use natural language to explain each step, incorporating LaTeX notation (e.g., $x + 2$) 
for mathematical expressions when helpful. Conclude your response with the final answer enclosed 
in a LaTeX \boxed{} environment (e.g., \boxed{5}). 
Place this at the end of your explanation as a standalone statement.

The question you should answer is: """



Finally, let's compare the agents output to our pseudo ground truth:

In [ ]:
response = agent.step(SYSTEM_PROMPT + obs.question)

proposed_solution = await extractor.extract(response.msgs[0].content)

result = await env.step(Action(index=0, llm_response=proposed_solution))

print(result)